# Playground

## Import Libraries

In [11]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot
from pathlib import Path

In [12]:
offline.init_notebook_mode(connected=True)

In [13]:
def save_chart(fig, chart_name):

    config = {'displaylogo': False,
            'responsive': True
            }

    name_fig = chart_name


    fig.write_image(
                    "../../../django_project/static/django_project/images/blog/{}.jpeg".format(
                        name_fig),
                    scale=1,
                    )
    plot(
        fig,
        config=config,
        filename = '../../../django_project/templates/django_project/blog/{}.html'.format(
            name_fig),
        auto_open=False)

# Compare CDP Scores and MSCi Implied Temperature Rating

In [14]:
path = Path(os.path.dirname (os.getcwd()))
XLSX_PATH = os.path.join(path.parent.parent.parent, 'sp100.xlsx')

## Compute total companies count and revenue by sector

In [19]:
# total number of companies
total_companies_count = 100

# total revenue of all companies

COLS_TO_USE = {
'companies':['company_id', 'company_name','SP100'],
'msci_itr':['company_id', 'msci_itr'],
'ghg_qual':['company_id', 'cdp_score', 'prev_cdp_score'],
'grouping':['company_id', 'Sector1'],
}

all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )
# Select Final Figures and Year 2019

cond1 = (merged_df['SP100'] == 1)

filter_cond = cond1 #& cond2
filter_cond = filter_cond

merged_df = merged_df.loc[filter_cond][['Sector1','company_name','company_id', 'msci_itr', 'cdp_score', 'prev_cdp_score']]

In [8]:

all_sectors = merged_df['Sector1'].unique()
all_scores = merged_df['cdp_score'].sort_values().unique()

In [7]:
import plotly.graph_objects as go
fig = go.Figure()
for sector in all_sectors:
    cdp_score = merged_df.loc[merged_df['Sector1'] == sector]['cdp_score']
    msci_itr = merged_df.loc[merged_df['Sector1'] == sector]['msci_itr']
    company_name = merged_df.loc[merged_df['Sector1'] == sector]['company_name']
    fig.add_trace(go.Scatter(
        x=cdp_score,
        y=msci_itr,
        text = company_name,
        name=sector,
        )
    )
fig.update_traces(
    mode='markers',
    marker=dict(
        line_width=1,
        symbol='circle', 
        size=20,
        ))

fig.update_xaxes(categoryorder='category ascending')


fig.update_layout(
    title="CDP vs. ITR<br><i>by net0tracker.org</i>",
    title_x=0.5,
    yaxis_title = "ITR (in °C)",
    xaxis_title = "CDP Score",
    xaxis=dict(
        showgrid=True,
        showline=True,
    ),
    #margin=dict(l=140, r=40, b=50, t=80),
    showlegend = False,
    legend=dict(
        #font_size=10,
        orientation="v",
        #yanchor="bottom",
        #y=-0.4,
        #xanchor="left",
        #x=0,
        title = "Sectors",
    ),
    #width=800,
    height=600,
    paper_bgcolor='white',
    #plot_bgcolor='white',
    hovermode='closest',
)
fig.show()

In [8]:
save_chart(fig, 'cdp_itr')

In [9]:
import plotly.express as px
df = merged_df
fig = px.histogram(
    df,
    x="msci_itr",
    marginal="rug",
    hover_name="company_name",
    title="Distribution of Implied Temperature Rise (ITR)<br><i>by net0tracker.org</i>",
    labels = {'msci_itr':'MSCi Temperature (in °C)'},
    color_discrete_sequence=['red'],
    cumulative=False,
    nbins = 25
    )

fig.update_traces(
    xbins=dict(
        start=1.21,
        end=4.5,
        size=0.10),
    selector = dict(type = 'histogram')
    )
fig.update_layout(
    title_x=0.5,
    yaxis_title = "Number of S&P100 companies",
    xaxis=dict(
        showgrid=False,
        #showline=True,
        #linecolor='rgb(102, 102, 102)',
        #tickfont_color='rgb(102, 102, 102)',
        #showticklabels=True,
        #dtick=10,
        #ticks='outside',
        #tickcolor='rgb(102, 102, 102)',
    ),
    margin=dict(l=140, r=40, b=50, t=80),
    width=800,
    height=600,
    paper_bgcolor='white',
    plot_bgcolor='white',
    hovermode='closest',
)

fig.show()

In [10]:
save_chart(fig,'itr_scores_dist')

In [20]:
df = merged_df
df['2021 CDP Score'] = df['cdp_score']
df['2020 CDP Score'] = df['prev_cdp_score']

In [22]:
import plotly.express as px
df = merged_df
fig = px.histogram(
    df,
    x=["2020 CDP Score",'2021 CDP Score'],
    hover_name="company_name",
    title="Distribution of 2021 CDP Scores vs. 2020<br><i>by net0tracker.org</i>",
    color_discrete_sequence=['orange','red'],
    barmode = 'group',
    cumulative=False,
    #nbins = 20,
    )
fig.update_layout(
    legend={
        "title": 'CDP Scores'
    })

fig.update_xaxes(categoryorder='category ascending')
fig.update_layout(
    title_x=0.5,
    yaxis_title = "Number of S&P100 companies",
    xaxis_title = "CDP Scores",
    xaxis=dict(
        showgrid=False,
    ),
    margin=dict(l=140, r=40, b=50, t=80),
    width=800,
    height=600,
    paper_bgcolor='white',
    plot_bgcolor='white',
    hovermode='closest',
)
fig.show()

In [23]:
save_chart(fig,'2021_cdp_scores_dist')